## This notebook joins together multiple data sources to find coverage for each block and block group

In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import sys
from IPython.display import clear_output

pd.set_option('display.max_rows', 210)
pd.set_option('display.max_columns', 210)

In [3]:
cpuc_zipfile     = "zip:///Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/Fixed_Consumer_Deployment_2020.zip"
census_excel     = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/Census_Blocks_2010.xlsx"
bc_csv           = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/BC_collapsed.csv"
telco_prof_csv   = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/telco_profiles.csv"
demographics_csv = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/BGdemographics2019.csv"
tri_csv          = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/tri_bg.csv"
ookla_csv        = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/ookla_bg.csv"

cpuc = gp.read_file(cpuc_zipfile)
cpuc_simple = cpuc[['BlockCode', 'DBA']]
cpuc_simple['BlockCode'] = cpuc_simple.BlockCode.astype('int64')

census = pd.read_excel(census_excel)
census = census.rename(columns = {"SE_T002_001":"Population", "SE_T002_002":"Population Density","SE_T002_006":"Area (sq mi)", "SE_T058_001":"Households"})

bc = pd.read_csv(bc_csv)
bc = bc.rename(columns = {"blockcode":"BlockCode", "blockgroup":"BlockGroup"})

telco_profiles = pd.read_csv(telco_prof_csv)

demographics = pd.read_csv(demographics_csv)
demographics = demographics.rename(columns = {'FIPS': 'BlockGroup'})
demographics = demographics.set_index('BlockGroup')

tri   = pd.read_csv(tri_csv)
tri   = tri.set_index("GEOID")

ookla = pd.read_csv(ookla_csv)
ookla = ookla.set_index("GEOID")
ookla = ookla[['avg_d_kbps_wt','avg_u_kbps_wt','avg_lat_ms_wt','devices','tests']]

<ipython-input-3-338baba6a909>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpuc_simple['BlockCode'] = cpuc_simple.BlockCode.astype('int64')


In [ ]:
census

In [4]:
# Find all rows in either a census place or in a census-designated urban area

# change value below to read from ua_place_cb_excel
read_ua_place_cb_excel = 1

ua_place_cb_excel        = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/ua_place_cb_2010.xlsx"
ua_demographics_excel    = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/ua_demographics.xlsx"
place_demographics_excel = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/place_demographics.xlsx"

if read_ua_place_cb_excel:
    ua_place_cb = pd.read_excel(ua_place_cb_excel)
    ua_place_cb['BlockCode'] = ua_place_cb.BlockCode.astype('int64')
    ua_place_cb = ua_place_cb.sort_values('BlockCode')
    ua_place_cb = ua_place_cb.set_index('BlockCode')
    ua_place_cb.drop_duplicates(inplace=True)
    ua_place_cb = ua_place_cb[ua_place_cb['CB_POP10']>0]
    
    ua_place_cb['is_urban'] = 0
    ua_place_cb['is_place'] = 0
    ua_place_cb['is_rural'] = 0

    ua_place_cb.loc[ua_place_cb['U_UACE10'].notnull() & ua_place_cb['P_GEOID10'].notnull(), ['is_urban', 'is_place']] = 1
    ua_place_cb.loc[ua_place_cb['U_UACE10'].notnull() & ua_place_cb['P_GEOID10'].isnull(),  'is_urban'] = 1
    ua_place_cb.loc[ua_place_cb['U_UACE10'].isnull()  & ua_place_cb['P_GEOID10'].notnull(), 'is_place'] = 1
    
ua_demographics    = pd.read_excel(ua_demographics_excel)

# lower bounds
a = 2500

# middle bounds
b = 10000

# upper bounds
c = 50000

ua_demographics.loc[ua_demographics['U_POP10']<a, 'U_Urban_Index'] = 1
ua_demographics.loc[(ua_demographics['U_POP10']>=a)  & (ua_demographics['U_POP10']<b), 'U_Urban_Index'] = 2
ua_demographics.loc[(ua_demographics['U_POP10']>=b) & (ua_demographics['U_POP10']<c), 'U_Urban_Index'] = 3
ua_demographics.loc[ua_demographics['U_POP10']>=c, 'U_Urban_Index'] = 4

place_demographics = pd.read_excel(place_demographics_excel)

place_demographics.loc[place_demographics['P_POP10']<a, 'P_Urban_Index'] = 1
place_demographics.loc[(place_demographics['P_POP10']>=a)  & (place_demographics['P_POP10']<b), 'P_Urban_Index'] = 2
place_demographics.loc[(place_demographics['P_POP10']>=b) & (place_demographics['P_POP10']<c), 'P_Urban_Index'] = 3
place_demographics.loc[place_demographics['P_POP10']>=c, 'P_Urban_Index'] = 4

#save the index
ua_place_cb['BlockCode'] = ua_place_cb.index

ua_place_cb = pd.merge(ua_place_cb, ua_demographics,    left_index=True, on='U_UACE10',  how='left')
ua_place_cb = pd.merge(ua_place_cb, place_demographics, left_index=True, on='P_GEOID10', how='left')

#take the minimum between the urban area urban_index and the census place urban_index
ua_place_cb['Urban_Index'] = ua_place_cb[['U_Urban_Index', 'P_Urban_Index']].min(axis=1)

ua_place_cb = ua_place_cb.set_index('BlockCode')

total_pop = census['Population'].sum()

place_4_pop = ua_place_cb.loc[ua_place_cb['P_Urban_Index']==4,'CB_POP10'].sum()
place_3_pop = ua_place_cb.loc[ua_place_cb['P_Urban_Index']==3,'CB_POP10'].sum()
place_2_pop = ua_place_cb.loc[ua_place_cb['P_Urban_Index']==2,'CB_POP10'].sum()
place_1_pop = total_pop - place_4_pop - place_3_pop - place_2_pop

ua_4_pop    = ua_place_cb.loc[ua_place_cb['U_Urban_Index']==4,'CB_POP10'].sum()
ua_3_pop    = ua_place_cb.loc[ua_place_cb['U_Urban_Index']==3,'CB_POP10'].sum()
ua_2_pop    = ua_place_cb.loc[ua_place_cb['U_Urban_Index']==2,'CB_POP10'].sum()
ua_1_pop    = total_pop - ua_4_pop - ua_3_pop - ua_2_pop

print("Place_1 Population: ", ua_place_cb.loc[ua_place_cb['P_Urban_Index']==1,'CB_POP10'].sum(), ", ", 100*place_1_pop/total_pop,"%")
print("Place_2 Population: ", ua_place_cb.loc[ua_place_cb['P_Urban_Index']==2,'CB_POP10'].sum(), ", ", 100*place_2_pop/total_pop,"%")
print("Place_3 Population: ", ua_place_cb.loc[ua_place_cb['P_Urban_Index']==3,'CB_POP10'].sum(), ", ", 100*place_3_pop/total_pop,"%")
print("Place_4 Population: ", ua_place_cb.loc[ua_place_cb['P_Urban_Index']==4,'CB_POP10'].sum(), ", ", 100*place_4_pop/total_pop,"%")
print('\n')
print("UA_1 Population:    ", ua_place_cb.loc[ua_place_cb['U_Urban_Index']==1,'CB_POP10'].sum(), ", ", 100*ua_1_pop/total_pop,"%")
print("UA_2 Population:    ", ua_place_cb.loc[ua_place_cb['U_Urban_Index']==2,'CB_POP10'].sum(), ", ", 100*ua_2_pop/total_pop,"%")
print("UA_3 Population:    ", ua_place_cb.loc[ua_place_cb['U_Urban_Index']==3,'CB_POP10'].sum(), ", ", 100*ua_3_pop/total_pop,"%")
print("UA_4 Population:    ", ua_place_cb.loc[ua_place_cb['U_Urban_Index']==4,'CB_POP10'].sum(), ", ", 100*ua_4_pop/total_pop,"%")

Place_1 Population:  518318 ,  16.10409106619442 %
Place_2 Population:  1699579 ,  4.562143682136737 %
Place_3 Population:  6912016 ,  18.553777214962082 %
Place_4 Population:  22642950 ,  60.77998803670676 %


UA_1 Population:     0 ,  14.296379691864134 %
UA_2 Population:     441240 ,  1.1844111267002086 %
UA_3 Population:     1409825 ,  3.78436319622002 %
UA_4 Population:     30076924 ,  80.73484598521564 %


In [5]:
ua_place_cb

,CB_HOUSING10,CB_POP10,U_UACE10,U_NAMELSAD10,P_GEOID10,P_NAMELSAD10,P_CLASSFP10,is_urban,is_place,is_rural,U_POP10,U_HU10,U_Urban_Index,P_POP10,P_Urban_Index,Urban_Index
BlockCode,,,,,,,,,,,,,,,,
60014001001007,56,117,78904.0,"San Francisco--Oakland, CA Urbanized Area",653000.0,Oakland city,C1,1,1,0,3281212.0,1338437.0,4.0,390724.0,4.0,4.0
60014001001008,10,20,78904.0,"San Francisco--Oakland, CA Urbanized Area",653000.0,Oakland city,C1,1,1,0,3281212.0,1338437.0,4.0,390724.0,4.0,4.0
60014001001011,6,13,78904.0,"San Francisco--Oakland, CA Urbanized Area",653000.0,Oakland city,C1,1,1,0,3281212.0,1338437.0,4.0,390724.0,4.0,4.0
60014001001013,3,1,78904.0,"San Francisco--Oakland, CA Urbanized Area",653000.0,Oakland city,C1,1,1,0,3281212.0,1338437.0,4.0,390724.0,4.0,4.0
60014001001015,14,33,78904.0,"San Francisco--Oakland, CA Urbanized Area",653000.0,Oakland city,C1,1,1,0,3281212.0,1338437.0,4.0,390724.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61150411004047,64,148,NaN,NaN,612612.0,Challenge-Brownsville CDP,U2,0,1,0,NaN,NaN,NaN,1148.0,1.0,1.0
61150411004050,5,13,NaN,NaN,612612.0,Challenge-Brownsville CDP,U2,0,1,0,NaN,NaN,NaN,1148.0,1.0,1.0
61150411004051,24,43,NaN,NaN,612612.0,Challenge-Brownsville CDP,U2,0,1,0,NaN,NaN,NaN,1148.0,1.0,1.0


In [ ]:
# Assign 0-1 Encoding for each ISP size based on household

telco_profiles = telco_profiles.fillna(0)

telco_profiles['local_telco_under_2000'] = 0
telco_profiles['local_telco_2000_4999']  = 0
telco_profiles['local_telco_5000_9999']  = 0
telco_profiles['local_telco_over_10000'] = 0

for DBA in telco_profiles['DBA'].index:
    if (telco_profiles.loc[DBA, 'local'] == 1):

        if telco_profiles.loc[DBA,'Households'] < 2000:
            telco_profiles.loc[DBA, 'local_telco_under_2000'] = 1

        elif telco_profiles.loc[DBA,'Households'] < 5000:
            telco_profiles.loc[DBA, 'local_telco_2000_4999']  = 1

        elif telco_profiles.loc[DBA,'Households'] < 10000:
            telco_profiles.loc[DBA, 'local_telco_5000_9999']  = 1

        else:
            telco_profiles.loc[DBA, 'local_telco_over_10000'] = 1

telco_profiles = telco_profiles.drop(['Population','Area (sq mi)','Households','Business_only','web'], axis = 1)

telco_profiles['isp_count'] = 0

In [ ]:
# Uncomment below to save telco_profiles
#telco_profiles.to_csv('telco_profiles_1.csv')
telco_profiles

In [ ]:
#Join cpuc_simple and census

telco_cols     = ['COLR','Small ILEC', 'CHCF_A','local','coop','private','muni',
                  'local_telco_under_2000','local_telco_2000_4999','local_telco_5000_9999','local_telco_over_10000', 'isp_count']

bc_groups_cols = ["BlockCode","BGpop","LAcounty",
                  "served","served25","served100","served_fib","comp","comp25","comp100","comp_fib","max_up","max_dn"]

joined_table = pd.merge(cpuc_simple, census, on='BlockCode', how='outer').drop_duplicates()
joined_table = joined_table.dropna(axis=0, subset=['DBA'])

joined_table = pd.merge(joined_table, telco_profiles[['DBA']+telco_cols], on='DBA', how = 'outer').drop_duplicates()
joined_table = pd.merge(joined_table, bc[bc_groups_cols], on='BlockCode', how='outer').drop_duplicates()
joined_table = joined_table.dropna(axis=0, subset=['DBA'])

joined_table = joined_table.dropna(axis=0, subset=['BlockCode'])
joined_table['BlockCode'] = joined_table.BlockCode.astype('int64')
joined_table = joined_table.set_index('BlockCode')
joined_table = joined_table.sort_index()

# remove all telco cols from the joined table if the population is zero
joined_table.loc[joined_table['Population']==0, telco_cols] = 0

In [ ]:
#joined_table.fillna(0).to_csv('block_isp_combo.csv')
joined_table.head(100)

In [ ]:
bc_detailed = pd.DataFrame(columns=telco_cols, index = joined_table.index.unique())
bc_detailed = bc_detailed.fillna(0)

count = 1
n_blocks = len(bc_detailed.index.unique())

# Find which blocks are served by each kind of ISP
for block_code in bc_detailed.index:
    print(count, ' / ', n_blocks)
    
    temp = joined_table.loc[[block_code]]

    temp = temp[telco_cols].max()
    
    temp['isp_count'] = len(joined_table.loc[[block_code]]['DBA'].unique())
    
    bc_detailed.loc[block_code,telco_cols] = temp
    
    count += 1
    clear_output(wait=True)

In [ ]:
#bc_detailed.to_csv("BC_collapsed_detailed.csv")
bc_detailed

In [ ]:
# Prepare bc data frame for urban index classification
ua_place_cb = ua_place_cb[['is_urban','is_place','is_rural','Urban_Index']]
bc = pd.merge(bc, ua_place_cb, on='BlockCode', how='left')
bc.loc[bc['Urban_Index'].isnull(), 'Urban_Index'] = 1
bc.loc[bc['is_urban'].isnull(), 'is_urban'] = 0;
bc.loc[bc['is_place'].isnull(), 'is_place'] = 0;
bc.loc[bc['is_rural'].isnull(), 'is_rural'] = 1;

In [ ]:
bc_1

In [ ]:
# Create the block group aggregate dataframe

# definitions for rural vs urban using urban_index column
# urban_index values:
# 1 - rural:      Majority of block group outside census designated urban areas or census designated places. Or inside a census designated urban areas or census designated places with a population less than 2500.
# 2 - small town: Majority of block group is inside either a census designated urban area or census place and has a population between 2,500 and 9,999
# 3 - city:       Majority of block group is inside either a census designated urban area or census place and has a population between 10,000 and 49,999
# 4 - urban area: Majority of block group is inside either a census designated urban area or census place and has a population greater than 50,000

telco_cols     = ['has_local','has_coop','has_private','has_muni','avg_local','avg_coop','avg_private','avg_muni',
                  'local_telco_under_2000','local_telco_2000_4999','local_telco_5000_9999','local_telco_over_10000', 'isp_count']

urban_index_cols = ["shrpop10_ui_1","shrpop10_ui_2","shrpop10_ui_3","shrpop10_ui_4","urban_index"]

bc_groups_cols = ["BGpop","LAcounty",
                  "served","served25","served100","served_fib","served_un","served_under25","served_under100",
                  "comp","comp25","comp100","comp_fib","comp_un","comp_under25","comp_under100", 
                  "max_up","max_dn","avg_max_up","avg_max_dn"] + telco_cols + urban_index_cols

bc_groups = pd.DataFrame(columns=bc_groups_cols, index = bc['BlockGroup'].unique())

bc_1 = (bc.set_index('BlockCode')).join(bc_detailed,  how='outer').fillna(0)

count = 1
n_groups = len(bc_groups.index)

# Group each block together
for group in bc_groups.index:
    
    print(count, ' / ', n_groups)
    
    if pd.isnull(group) is None:
        continue
    
    cond = (bc_1['BlockGroup']==group)
    temp = bc_1.loc[cond]
    
    n_blocks = len(temp)
    
    bc_groups["BGpop"].loc[group]           = temp["BGpop"].iloc[0]
    bc_groups["LAcounty"].loc[group]        = temp["LAcounty"].iloc[0]
    
    # Information about service coverage in general
    bc_groups["served"].loc[group]          = (temp["served"]*temp['shrpop10']).sum()
    bc_groups["served25"].loc[group]        = (temp["served25"]*temp['shrpop10']).sum()
    bc_groups["served100"].loc[group]       = (temp["served100"]*temp['shrpop10']).sum()
    bc_groups["served_fib"].loc[group]      = (temp["served_fib"]*temp['shrpop10']).sum()
    bc_groups["served_un"].loc[group]       = 1.0 - bc_groups["served"].loc[group]
    bc_groups["served_under25"].loc[group]  = bc_groups["served"].loc[group] - bc_groups["served25"].loc[group]
    bc_groups["served_under100"].loc[group] = bc_groups["served"].loc[group] - bc_groups["served100"].loc[group]
    
    bc_groups["comp"].loc[group]            = (temp["comp"]*temp['shrpop10']).sum()
    bc_groups["comp25"].loc[group]          = (temp["comp25"]*temp['shrpop10']).sum()
    bc_groups["comp100"].loc[group]         = (temp["comp100"]*temp['shrpop10']).sum()
    bc_groups["comp_fib"].loc[group]        = (temp["comp_fib"]*temp['shrpop10']).sum()
    bc_groups["comp_un"].loc[group]         = 1.0 - bc_groups["comp"].loc[group]
    bc_groups["comp_under25"].loc[group]    = bc_groups["comp"].loc[group] - bc_groups["comp25"].loc[group]
    bc_groups["comp_under100"].loc[group]   = bc_groups["comp"].loc[group] - bc_groups["comp100"].loc[group]
    
    # absolute maximum and average maximum speeds for each block group
    bc_groups["max_up"].loc[group]          = temp["max_up"].max()
    bc_groups["max_dn"].loc[group]          = temp["max_dn"].max()
    bc_groups["avg_max_up"].loc[group]      = (temp["max_up"]*temp['shrpop10']).sum()
    bc_groups["avg_max_dn"].loc[group]      = (temp["max_dn"]*temp['shrpop10']).sum()
    
    # binary values regarding small ISP service coverage
    bc_groups["has_local"].loc[group]           = (temp["local"]).max()
    bc_groups["has_coop"].loc[group]            = (temp["coop"]).max()
    bc_groups["has_private"].loc[group]         = (temp["private"]).max()
    bc_groups["has_muni"].loc[group]            = (temp["muni"]).max()
    
    # weighted averages regarding small ISP service coverage
    bc_groups["avg_local"].loc[group]           = (temp["local"]*temp['shrpop10']).sum()
    bc_groups["avg_coop"].loc[group]            = (temp["coop"]*temp['shrpop10']).sum()
    bc_groups["avg_private"].loc[group]         = (temp["private"]*temp['shrpop10']).sum()
    bc_groups["avg_muni"].loc[group]            = (temp["muni"]*temp['shrpop10']).sum()
    
    # percentage of people reached by isps of each size
    bc_groups["local_telco_under_2000"].loc[group] = (temp["local_telco_under_2000"]*temp['shrpop10']).sum()
    bc_groups["local_telco_2000_4999"].loc[group]  = (temp["local_telco_2000_4999"]*temp['shrpop10']).sum()
    bc_groups["local_telco_5000_9999"].loc[group]  = (temp["local_telco_5000_9999"]*temp['shrpop10']).sum()
    bc_groups["local_telco_over_10000"].loc[group] = (temp["local_telco_over_10000"]*temp['shrpop10']).sum()
    
    # get the unique isp count in each block group
    joined_table_temp = joined_table.loc[cond, 'DBA']
    bc_groups["isp_count"].loc[group] = len(joined_table_temp.unique())

    # percentage of people in each urban_index group
    # take the largest value to calculate the urban index for the entire block group
    bc_groups["shrpop10_ui_1"].loc[group] = (temp["Urban_Index"].apply(lambda x: 1 if x==1 else 0)*temp['shrpop10']).sum()
    bc_groups["shrpop10_ui_2"].loc[group] = (temp["Urban_Index"].apply(lambda x: 1 if x==2 else 0)*temp['shrpop10']).sum()
    bc_groups["shrpop10_ui_3"].loc[group] = (temp["Urban_Index"].apply(lambda x: 1 if x==3 else 0)*temp['shrpop10']).sum()
    bc_groups["shrpop10_ui_4"].loc[group] = (temp["Urban_Index"].apply(lambda x: 1 if x==4 else 0)*temp['shrpop10']).sum()
    
    largest_index_group   = pd.to_numeric(bc_groups.loc[group,['shrpop10_ui_1', 'shrpop10_ui_2', 'shrpop10_ui_3', 'shrpop10_ui_4']]).idxmax(axis=1)
    
    if largest_index_group=='shrpop10_ui_1':
        bc_groups["urban_index"].loc[group] = 1
        
    elif largest_index_group=='shrpop10_ui_2':
        bc_groups["urban_index"].loc[group] = 2
        
    elif largest_index_group=='shrpop10_ui_3':
        bc_groups["urban_index"].loc[group] = 3
        
    elif largest_index_group=='shrpop10_ui_4':
        bc_groups["urban_index"].loc[group] = 4
    
    count+=1
    clear_output(wait=True)

In [ ]:
bc_groups[(bc_groups['has_local']==1) & (bc_groups['isp_count']==6)]

In [ ]:
print("Urban Index 1 Population: ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_1"]).sum(), ", ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_1"]).sum()/bc_groups["BGpop"].sum()*100, "%")
print("Urban Index 2 Population: ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_2"]).sum(), ", ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_2"]).sum()/bc_groups["BGpop"].sum()*100, "%")
print("Urban Index 3 Population: ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_3"]).sum(), ", ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_3"]).sum()/bc_groups["BGpop"].sum()*100, "%")
print("Urban Index 4 Population: ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_4"]).sum(), ", ", (bc_groups["BGpop"] * bc_groups["shrpop10_ui_4"]).sum()/bc_groups["BGpop"].sum()*100, "%")

In [ ]:
# merge the bc_groups dataframe with the demographics, terrain ruggedness index (tri), and ookla dataframes on the block group code

merged1 = pd.merge(bc_groups, demographics, left_index=True, right_index=True, how='left').drop_duplicates()

merged2 = pd.merge(merged1, tri, left_index=True, right_index=True, how='left').drop_duplicates()

merged3 = pd.merge(merged2, ookla, left_index=True, right_index=True, how='left').drop_duplicates()

print(merged3.shape)
merged3.head(100)

merged3.to_excel('BC_collapsed_by_group_detailed.xlsx')

In [3]:
# read the 'BC_collapsed_by_group_detailed.xlsx' file
# prepare it for cem processing
data = pd.read_excel('BC_collapsed_by_group_detailed.xlsx', index_col=0)

In [4]:
data.columns

Index(['BGpop', 'LAcounty', 'served', 'served25', 'served100', 'served_fib',
       'served_un', 'served_under25', 'served_under100', 'comp', 'comp25',
       'comp100', 'comp_fib', 'comp_un', 'comp_under25', 'comp_under100',
       'max_up', 'max_dn', 'avg_max_up', 'avg_max_dn', 'has_local', 'has_coop',
       'has_private', 'has_muni', 'avg_local', 'avg_coop', 'avg_private',
       'avg_muni', 'local_telco_under_2000', 'local_telco_2000_4999',
       'local_telco_5000_9999', 'local_telco_over_10000', 'isp_count',
       'shrpop10_ui_1', 'shrpop10_ui_2', 'shrpop10_ui_3', 'shrpop10_ui_4',
       'urban_index', 'GEOID', 'NAME', 'COUNTY', 'TRACT', 'BLKGRP',
       'A00001_001', 'A00002_002', 'A00002_003', 'A10003_001', 'A14006_001',
       'A14008_001', 'A10036_001', 'B28002001', 'white', 'black', 'amindian',
       'asian', 'native', 'hispanic', 'laborforce16', 'children', 'unemployed',
       'povertyrate', 'medianage', 'lessHS', 'bachelor_more', 'nointernet',
       'has_pc', 'has_pca

In [5]:
columns_for_data_analysis = ['served', 'served25', 'served100', 'served_fib', 
                             'comp', 'comp25', 'comp100', 'comp_fib', 
                             'max_up', 'max_dn', 'avg_max_up', 'avg_max_dn', 
                             'has_coop', 'has_private', 'has_muni', 
                             'avg_local', 'avg_coop', 'avg_private', 'avg_muni', 
                             'local_telco_under_2000', 'local_telco_2000_4999', 'local_telco_5000_9999', 'local_telco_over_10000', 'isp_count',
                             'shrpop10_ui_1', 'shrpop10_ui_2', 'shrpop10_ui_3', 'shrpop10_ui_4', 'urban_index',
                             'A00001_001', 'A00002_002', 'A00002_003', 'A10003_001', 'A14006_001', 'A14008_001', 'A10036_001', 'B28002001', 
                             'white', 'black', 'amindian', 'asian', 'native', 'hispanic', 
                             'laborforce16', 'unemployed', 'povertyrate', 'medianage', 'lessHS', 'bachelor_more', 
                             'nointernet', 'has_pc', 'has_pcandbroadband', 'has_internet', 'has_broadband', 'has_mobileonly', 'has_cabledslfib', 
                             'tri', 
                             'avg_d_kbps_wt', 'avg_u_kbps_wt', 'avg_lat_ms_wt']

data = data[['has_local'] + columns_for_data_analysis]
data = data.drop(data[(data['has_local']==1) & (data['isp_count']==1)].index)
data.to_excel('BC_collapsed_by_group_detailed_for_r_cem.xlsx')